### Cumulative stats up to the year before to predict the following years salary 

### Regression - linear, decision, neural networks

### Classification - extend contract or release player

In [24]:
import requests
import json
import pandas as pd
import pickle
import numpy as np
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', 999)


import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.insert(0, 'src/')
from data_cleaning import *

In [2]:
# nhl_teams = {}

# for team in range(1,102):
    
#     url = f"https://statsapi.web.nhl.com/api/v1/teams/{team}"

#     headers = {
#         'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
#         'x-rapidapi-host': "api-hockey.p.rapidapi.com"
#     }

#     nhl_teams[team] = requests.request("GET", url, headers=headers)


In [3]:
# errors_404 = [59, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 83, 84, 85, 86, 99]

# for i in errors_404:
#     nhl_teams.pop(i)

In [4]:
# pickle_out = open("json_files/nhl_teams.pickle","wb")
# pickle.dump(nhl_teams, pickle_out)
# pickle_out.close()

pickle_in = open("json_files/nhl_teams.pickle","rb")
nhl_teams = pickle.load(pickle_in)

all_teams_df = data_cleaning.get_teams(nhl_teams)
all_teams_df.head()

,id,name,link,venue_name,venue_link,venue_city,venue_timeZone,abbreviation,teamName,locationName,firstYearOfPlay,division_id,division_name,division_link,conference_id,conference_name,conference_link,franchise_franchiseId,franchise_teamName,franchise_link,shortName,officialSiteUrl,franchiseId,active,venue_id,division_nameShort,division_abbreviation
1,1,New Jersey Devils,/api/v1/teams/1,Prudential Center,/api/v1/venues/null,Newark,America/New_York,NJD,Devils,New Jersey,1982,25,MassMutual East,/api/v1/divisions/25,6,Eastern,/api/v1/conferences/6,23,Devils,/api/v1/franchises/23,New Jersey,http://www.newjerseydevils.com/,23,True,NaN,NaN,NaN
2,2,New York Islanders,/api/v1/teams/2,Nassau Veterans Memorial Coliseum,/api/v1/venues/null,Uniondale,America/New_York,NYI,Islanders,New York,1972,25,MassMutual East,/api/v1/divisions/25,6,Eastern,/api/v1/conferences/6,22,Islanders,/api/v1/franchises/22,NY Islanders,http://www.newyorkislanders.com/,22,True,NaN,NaN,NaN
3,3,New York Rangers,/api/v1/teams/3,Madison Square Garden,/api/v1/venues/5054,New York,America/New_York,NYR,Rangers,New York,1926,25,MassMutual East,/api/v1/divisions/25,6,Eastern,/api/v1/conferences/6,10,Rangers,/api/v1/franchises/10,NY Rangers,http://www.newyorkrangers.com/,10,True,5054,NaN,NaN
4,4,Philadelphia Flyers,/api/v1/teams/4,Wells Fargo Center,/api/v1/venues/5096,Philadelphia,America/New_York,PHI,Flyers,Philadelphia,1967,25,MassMutual East,/api/v1/divisions/25,6,Eastern,/api/v1/conferences/6,16,Flyers,/api/v1/franchises/16,Philadelphia,http://www.philadelphiaflyers.com/,16,True,5096,NaN,NaN
5,5,Pittsburgh Penguins,/api/v1/teams/5,PPG Paints Arena,/api/v1/venues/5034,Pittsburgh,America/New_York,PIT,Penguins,Pittsburgh,1967,25,MassMutual East,/api/v1/divisions/25,6,Eastern,/api/v1/conferences/6,17,Penguins,/api/v1/franchises/17,Pittsburgh,http://pittsburghpenguins.com/,17,True,5034,NaN,NaN


In [5]:
# nhl_team_stadiums = {}

# for link in all_teams_df['venue_link']:

#     url = f"https://statsapi.web.nhl.com/api/v1/teams/{link}/roster"

#     headers = {
#         'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
#         'x-rapidapi-host': "api-hockey.p.rapidapi.com"
#     }

#     nhl_team_stadiums[link] = requests.request("GET", url, headers=headers)

print('Response [404]')

Response [404]


In [6]:
# nhl_team_franchise = {}

# for link in all_teams_df['franchise_link']:

#     url = f"https://statsapi.web.nhl.com/api/v1/teams/{link}/roster"

#     headers = {
#         'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
#         'x-rapidapi-host': "api-hockey.p.rapidapi.com"
#     }

#     nhl_team_franchise[link] = requests.request("GET", url, headers=headers)

print('Response [404]')

Response [404]


In [7]:
# nhl_team_roster = {}

# for number in range(1,55):

#     url = f"https://statsapi.web.nhl.com/api/v1/teams/{number}/roster"

#     headers = {
#         'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
#         'x-rapidapi-host': "api-hockey.p.rapidapi.com"
#     }

#     nhl_team_roster[number] = requests.request("GET", url, headers=headers)

In [8]:
# roster_error_404 = [11, 27, 31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]
# for error in roster_error_404:
#     nhl_team_roster.pop(error)

In [9]:
# pickle_out = open("json_files/nhl_rosters.pickle","wb")
# pickle.dump(nhl_team_roster, pickle_out)
# pickle_out.close()

In [10]:
pickle_in = open("json_files/nhl_rosters.pickle","rb")
nhl_rosters = pickle.load(pickle_in)

In [11]:
rosters = data_cleaning.build_roster(nhl_rosters)

In [12]:
rosters

,id,fullName,link,jerseyNumber,code,name,type,abbreviation,Team_Number
0,8477972,Josh Jacobs,/api/v1/people/8477972,40,D,Defenseman,Defenseman,D,1
1,8478421,A.J. Greer,/api/v1/people/8478421,42,L,Left Wing,Forward,LW,1
2,8474056,P.K. Subban,/api/v1/people/8474056,76,D,Defenseman,Defenseman,D,1
3,8475809,Scott Wedgewood,/api/v1/people/8475809,41,G,Goalie,Goalie,G,1
4,8476807,Matt Tennyson,/api/v1/people/8476807,7,D,Defenseman,Defenseman,D,1
...,...,...,...,...,...,...,...,...,...
941,8479639,Dylan Coghlan,/api/v1/people/8479639,52,D,Defenseman,Defenseman,D,54
942,8479974,Nolan Patrick,/api/v1/people/8479974,19,C,Center,Forward,C,54
943,8479980,Nicolas Hague,/api/v1/people/8479980,14,D,Defenseman,Defenseman,D,54
944,8480313,Logan Thompson,/api/v1/people/8480313,36,G,Goalie,Goalie,G,54


In [13]:
# player_stats = {}
# for link in rosters['link']:

#     url = f"https://statsapi.web.nhl.com/{link}"

#     headers = {
#         'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
#         'x-rapidapi-host': "api-hockey.p.rapidapi.com"
#     }

#     player_stats[link] = requests.request("GET", url, headers=headers)

In [14]:
# pickle_out = open("json_files/nhl_player_stats.pickle","wb")
# pickle.dump(player_stats, pickle_out)
# pickle_out.close()

In [15]:
pickle_in = open("json_files/nhl_player_stats.pickle","rb")
nhl_player_stats = pickle.load(pickle_in)

In [16]:
player_stats = data_cleaning.player_statistics(rosters, nhl_player_stats)
player_stats.head()

,id,fullName,link,jerseyNumber,code,name,type,abbreviation,Team_Number,index,id,fullName,link,firstName,lastName,primaryNumber,birthDate,currentAge,birthCity,birthStateProvince,birthCountry,nationality,height,weight,active,alternateCaptain,captain,rookie,shootsCatches,rosterStatus,current_team_id,current_team_name,current_team_link,primary_position_code,primary_position_name,primary_position_type,primary_position_abbreviation
0,8477972,Josh Jacobs,/api/v1/people/8477972,40,D,Defenseman,Defenseman,D,1,/api/v1/people/8477972,8477972,Josh Jacobs,/api/v1/people/8477972,Josh,Jacobs,40,1996-02-15,25,Shelby Township,MI,USA,USA,"6' 2""",200,True,False,False,True,R,Y,1,New Jersey Devils,/api/v1/teams/1,D,Defenseman,Defenseman,D
1,8478421,A.J. Greer,/api/v1/people/8478421,42,L,Left Wing,Forward,LW,1,/api/v1/people/8478421,8478421,A.J. Greer,/api/v1/people/8478421,A.J.,Greer,42,1996-12-14,24,Joliette,QC,CAN,CAN,"6' 3""",210,True,False,False,False,L,Y,1,New Jersey Devils,/api/v1/teams/1,L,Left Wing,Forward,LW
2,8474056,P.K. Subban,/api/v1/people/8474056,76,D,Defenseman,Defenseman,D,1,/api/v1/people/8474056,8474056,P.K. Subban,/api/v1/people/8474056,P.K.,Subban,76,1989-05-13,32,Toronto,ON,CAN,CAN,"6' 0""",210,True,True,False,False,R,Y,1,New Jersey Devils,/api/v1/teams/1,D,Defenseman,Defenseman,D
3,8475809,Scott Wedgewood,/api/v1/people/8475809,41,G,Goalie,Goalie,G,1,/api/v1/people/8475809,8475809,Scott Wedgewood,/api/v1/people/8475809,Scott,Wedgewood,41,1992-08-14,28,Brampton,ON,CAN,CAN,"6' 2""",207,True,False,False,False,L,Y,1,New Jersey Devils,/api/v1/teams/1,G,Goalie,Goalie,G
4,8476807,Matt Tennyson,/api/v1/people/8476807,7,D,Defenseman,Defenseman,D,1,/api/v1/people/8476807,8476807,Matt Tennyson,/api/v1/people/8476807,Matt,Tennyson,7,1990-04-23,31,Minneapolis,MN,USA,USA,"6' 2""",205,True,False,False,False,R,Y,1,New Jersey Devils,/api/v1/teams/1,D,Defenseman,Defenseman,D


In [17]:
# player_stats_20202021 = data_cleaning.player_stats_by_year(rosters, '20202021' )
# player_stats_20192020 = data_cleaning.player_stats_by_year(rosters, '20192020' )
# player_stats_20182019 = data_cleaning.player_stats_by_year(rosters, '20182019' )
# player_stats_20172018 = data_cleaning.player_stats_by_year(rosters, '20172018' )
# player_stats_20162017 = data_cleaning.player_stats_by_year(rosters, '20162017' )
# player_stats_20152016 = data_cleaning.player_stats_by_year(rosters, '20152016' )
# player_stats_20142015 = data_cleaning.player_stats_by_year(rosters, '20142015' )
# player_stats_20132014 = data_cleaning.player_stats_by_year(rosters, '20132014' )

In [18]:
# pickle_out = open("json_files/nhl_player_stats20132014.pickle","wb")
# pickle.dump(player_stats_20132014, pickle_out)
# pickle_out.close()

# pickle_out = open("json_files/nhl_player_stats20142015.pickle","wb")
# pickle.dump(player_stats_20142015, pickle_out)
# pickle_out.close()

# pickle_out = open("json_files/nhl_player_stats20152016.pickle","wb")
# pickle.dump(player_stats_20152016, pickle_out)
# pickle_out.close()

# pickle_out = open("json_files/nhl_player_stats20162017.pickle","wb")
# pickle.dump(player_stats_20162017, pickle_out)
# pickle_out.close()

# pickle_out = open("json_files/nhl_player_stats20172018.pickle","wb")
# pickle.dump(player_stats_20172018, pickle_out)
# pickle_out.close()

# pickle_out = open("json_files/nhl_player_stats20182019.pickle","wb")
# pickle.dump(player_stats_20182019, pickle_out)
# pickle_out.close()

# pickle_out = open("json_files/nhl_player_stats20192020.pickle","wb")
# pickle.dump(player_stats_20192020, pickle_out)
# pickle_out.close()

# pickle_out = open("json_files/nhl_player_stats20202021.pickle","wb")
# pickle.dump(player_stats_20202021, pickle_out)
# pickle_out.close()

In [19]:
pickle_in = open("json_files/nhl_player_stats20132014.pickle","rb")
player_stats_20132014 = pickle.load(pickle_in)

pickle_in = open("json_files/nhl_player_stats20142015.pickle","rb")
player_stats_20142015 = pickle.load(pickle_in)

pickle_in = open("json_files/nhl_player_stats20152016.pickle","rb")
player_stats_20152016 = pickle.load(pickle_in)

pickle_in = open("json_files/nhl_player_stats20162017.pickle","rb")
player_stats_20162017 = pickle.load(pickle_in)

pickle_in = open("json_files/nhl_player_stats20172018.pickle","rb")
player_stats_20172018 = pickle.load(pickle_in)

pickle_in = open("json_files/nhl_player_stats20182019.pickle","rb")
player_stats_20182019 = pickle.load(pickle_in)

pickle_in = open("json_files/nhl_player_stats20192020.pickle","rb")
player_stats_20192020 = pickle.load(pickle_in)

pickle_in = open("json_files/nhl_player_stats20202021.pickle","rb")
player_stats_20202021 = pickle.load(pickle_in)

In [26]:
play_stats_20202021 = extract_player_stats(rosters, player_stats_20202021)
play_stats_20192020 = extract_player_stats(rosters, player_stats_20192020)
play_stats_20182019 = extract_player_stats(rosters, player_stats_20182019)
play_stats_20172018 = extract_player_stats(rosters, player_stats_20172018)
play_stats_20162017 = extract_player_stats(rosters, player_stats_20162017)
play_stats_20152016 = extract_player_stats(rosters, player_stats_20152016)
play_stats_20142015 = extract_player_stats(rosters, player_stats_20142015)
play_stats_20132014 = extract_player_stats(rosters, player_stats_20132014)

# Web Scraping

In [29]:
all_teams_df_active = all_teams_df[all_teams_df['active'] == True]

team_names = []
for name in all_teams_df_active['name']:
    team_names.append(name.lower().replace(" ", "-"))
    
team_names = team_names[:31]
team_names

['new-jersey-devils',
 'new-york-islanders',
 'new-york-rangers',
 'philadelphia-flyers',
 'pittsburgh-penguins',
 'boston-bruins',
 'buffalo-sabres',
 'montréal-canadiens',
 'ottawa-senators',
 'toronto-maple-leafs',
 'carolina-hurricanes',
 'florida-panthers',
 'tampa-bay-lightning',
 'washington-capitals',
 'chicago-blackhawks',
 'detroit-red-wings',
 'nashville-predators',
 'st.-louis-blues',
 'calgary-flames',
 'colorado-avalanche',
 'edmonton-oilers',
 'vancouver-canucks',
 'anaheim-ducks',
 'dallas-stars',
 'los-angeles-kings',
 'san-jose-sharks',
 'columbus-blue-jackets',
 'minnesota-wild',
 'winnipeg-jets',
 'arizona-coyotes',
 'vegas-golden-knights']

In [30]:
stats_link = []

for team in team_names:

    url = f'https://www.spotrac.com/nhl/{team}/cap'

    spotrac = requests.get(url)

    soup = BeautifulSoup(spotrac.text, 'html.parser')

    base_url = 'https://www.spotrac.com/redirect'

    for link in soup.find_all('a'):

        url = link.get('href')

        if url and '/player/' in url:

            stats_link.append(url)

In [ ]:
known_bad = []

known_good = []

for i in stats_link:
        
    url = i + 'cash-earnings/'
    
    test_request = requests.get(url)
    
    soup = BeautifulSoup(test_request.text, 'lxml')

    try:    
    
        testing = soup.body.find('div', class_='teams').find('ul', class_='cd-tabs-content')\
        .li.section.table.get_text().split()

    except:

        known_bad.append(i)

    name =[soup.h1.text]

    for i in testing:
        name.append(i)

    known_good.append(name)